# Imports and setup

In [1]:
!pip install mtcnn

# Import libraries
import cv2
from mtcnn import MTCNN
import numpy as np
from IPython.display import HTML
from base64 import b64encode

In [9]:
# Obtain Haar Cascade Classifier
!wget https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_frontalface_default.xml

In [ ]:
# Get sample video
!wget https://cdn.pixabay.com/videvo_files/video/2016-03/large_mp4/india-people-asia-street-asian-1643.mp4 -O india_people.mp4

# Video processing & Face detection in Python using OpenCV

In [ ]:
# Path to video file
video_path = "video.mp4"
cap = cv2.VideoCapture(video_path)

# Video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Initialize VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video_mtcnn.mp4', fourcc, fps, (width, height))

# Initialize detector MTCNN
detector = MTCNN()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB because MTCNN expects RGB,
    # not BGR like the format OpenCV uses
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces in the frames
    faces = detector.detect_faces(rgb_frame)

    # Draw rectangles around faces
    for face in faces:
        x, y, w, h = face['box']
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # Display confidence score
        confidence = face['confidence']
        cv2.putText(frame, f"{confidence:.2f}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    out.write(frame)

    # Real time output in Colab:
    from google.colab.patches import cv2_imshow
    cv2_imshow(frame)
    cv2.waitKey(1)

cap.release()
out.release()

mp4 = open('output_video_mtcnn.mp4', 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=640 controls>
  <source src="%s" type="video/mp4">
</video>
""" % data_url)

# Video processing & Face detection in C++ using OpenCV

In [4]:
%%writefile main.cpp
#include <opencv2/opencv.hpp>
#include <opencv2/objdetect.hpp>
#include <iostream>

using namespace cv;
using namespace std;

int main() {
    // Load Haar Cascade classifier
    CascadeClassifier faceCascade;
    if (!faceCascade.load("haarcascade_frontalface_default.xml")) {
        cerr << "Error loading Haar Cascade classifier!" << endl;
        return -1;
    }

    // Open input video
    // Modify name/ path if needed
    VideoCapture cap("video.mp4");
    if (!cap.isOpened()) {
        cerr << "Couldn't open input video!" << endl;
        return -1;
    }

    // Obtain video properties
    int width = static_cast<int>(cap.get(CAP_PROP_FRAME_WIDTH));
    int height = static_cast<int>(cap.get(CAP_PROP_FRAME_HEIGHT));
    double fps = cap.get(CAP_PROP_FPS);

    // Initialize VideoWriter to save final results
    VideoWriter writer("output_video_haar_cpp.mp4", VideoWriter::fourcc('m','p','4','v'), fps, Size(width, height));
    if (!writer.isOpened()){
         cerr << "Couldn't open output video!" << endl;
         return -1;
    }

    Mat frame;
    while (cap.read(frame)) {
        // Convert frame to grayscale for faster detection
        Mat gray;
        cvtColor(frame, gray, COLOR_BGR2GRAY);
        equalizeHist(gray, gray);

        // Detect faces in image/ frame
        vector<Rect> faces;
        faceCascade.detectMultiScale(gray, faces, 1.1, 3, 0, Size(30, 30));

        // Draw rectangles
        for (size_t i = 0; i < faces.size(); i++) {
            rectangle(frame, faces[i], Scalar(0, 255, 0), 2);
        }

        // Write frame in output video file
        writer.write(frame);
    }

    cap.release();
    writer.release();
    cout << "Video processing ended successfully!" << endl;
    return 0;
}

Overwriting main.cpp


In [5]:
!g++ main.cpp -o main `pkg-config --cflags --libs opencv4`

In [ ]:
!./main

In [7]:
!file video.mp4

# check if video exists

video.mp4: ISO Media, MP4 v2 [ISO 14496-14]
